In [32]:
import psycopg2
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

In [54]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [30]:
conn.close()
cursor.close()

In [23]:
req = """SELECT DISTINCT
content_words.word_id,content_words.word_lemma, content_words.word_value
FROM content_words
WHERE content_words.word_value !~ ('\W') AND
(array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
and word_lemma != 0 and word_hash = calc_hash('mother')"""
cursor.execute(req)
a = cursor.fetchone()

if not a:
    print("huyok")
else:
    print(a[0])

26408


In [57]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not a:
        print(word, " is not in db")
        return 0
    else:
        #print(a[0])
        word_id = a[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    for word in cursor:
        print(word[0])
get_word_forms("book")


mothers
mothering
motherin
mothered


In [38]:
def get_definitions(word, from_lang, to_lang):

    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    first_string_passed = False
    translations_variants = []
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            print(sence_word)
            translations_variants.append(sence_word)
        first_string_passed = True
    time.sleep(2)
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        print("========================================")
        print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + word + "\w{0,2}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                print(example)
    return translations_variants
 
many_word_war = get_definitions("mother", "русский", "английский")



мать
матерь
мама
матушка
мамочка
мамаша
мам
госпожа
матерей
материнства
родному
материнских

EXAMPLES FOR SENCE  мать 

The custody of illegitimate children remains exclusively to the mother and her relations.
If you upset your mother, you'll be in trouble.
I've only been back once, to see my mother.
It felt good to be holding the mother of my child.
I think your mother manipulated the news for her own marketing.
It's like screwing the daughter and the mother at the same time.
You're quite lucky to have a mother who looks out for you.
And my mother was a kind of a designer, also.
Five days later, the mother of the girl received an anonymous phone call confirming the abduction.
His mother had to leave Azerbaijan, leaving her son behind with his father.
According to the law, the child has but one recognized parent: the mother.
Therefore, the mother can lose the right to care for her children.
As a result, the author' mother was fined.
When the mother is healthy, the well-being of a famil


EXAMPLES FOR SENCE  материнских 

If you want to maternal feelings, so Go talk to mother in Stuttgart.


In [39]:
many_word_war

['мать',
 'матерь',
 'мама',
 'матушка',
 'мамочка',
 'мамаша',
 'мам',
 'госпожа',
 'матерей',
 'материнства',
 'родному',
 'материнских']

In [40]:
url = "https://dictionary.cambridge.org/dictionary/english/mothering"
#login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8' 
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   MOTHERING | meaning in the Cambridge English Dictionary
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="mothering definition: the process of caring for children as their mother or of caring for people in the way that a mother does: . Learn more." name="description"/>
  <meta content="mothering definition, dictionary, english, british, american, business, british english, thesaurus, define mothering, mothering meaning, what is mothering, spelling, conjugation, audio pronunciation, free, online, english." name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
  <meta content="https://dictionary.cambridge.org/dictionary/english/mothering" property="og:url"/>
  <link href="https://dictionary.cambridge.org/dictio

In [52]:
def get_wordform_def_info(word, from_lang, multisence_dict):
    multisence_dict[word] = []
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "/" + word + "_" + str(index)
        print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                #print(link.find("span", attrs = {'class':"trans"}))
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    print("ENG_DEFIN_EX", ex.text.strip())
                print(current_definition)
                multisence_dict[word].append(current_definition)
                current_definition = {}
                print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_wordform_def_info("mothering", "english",sence_dict)


https://dictionary.cambridge.org/dictionary/english/mothering_1
DEFINITION the process of caring for children as their mother or of caring for people in the way that a mother does:
ENG_DEFIN_EX He did not receive good mothering as a child.
ENG_DEFIN_EX a woman's mothering instinct
{'definition': 'the process of caring for children as their mother or of caring for people in the way that a mother does:', 'examples': ['He did not receive good mothering as a child.', "a woman's mothering instinct"]}
===
https://dictionary.cambridge.org/dictionary/english/mothering_2
